# Clean and Merge all data




In [1]:
# Dependencies and Setup
import json
import os
import pandas as pd
import urllib.request
import requests
from sqlalchemy import create_engine

## Load mobility data


In [2]:
csv_file = "Resources/data_us.csv"
data_us = pd.read_csv(csv_file)
data_us.head()

,Unnamed: 0,states,dates,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential,SMA_retail_recreation,SMA_grocery_pharmacy,SMA_parks,SMA_transit,SMA_workplaces,SMA_residential
0,0,Alabama,2020-02-15,5.155172,0.230769,49.571429,5.761905,0.784314,-0.863636,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667
1,1,Alabama,2020-02-16,-1.454545,-4.015625,-5.476190,2.136364,-1.920000,0.523810,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333
2,2,Alabama,2020-02-17,-1.169492,-4.476190,12.933333,5.727273,-16.264706,4.333333,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667
3,3,Alabama,2020-02-18,-4.280702,-5.000000,-12.466667,-0.681818,0.338235,1.435897,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667
4,4,Alabama,2020-02-19,1.438596,-0.730159,6.666667,3.714286,1.397059,0.000000,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667


## Load daily Covid cases data

In [3]:
csv_file = "Resources/state covid_data.csv"
daily_covid = pd.read_csv(csv_file)
daily_covid.rename(columns={"state_name":"states","date": "dates"}, inplace =True)
daily_covid.head()

,statefips,case_count,new_case_count,states,dates
0,1,0,0.0,Alabama,2020-02-01
1,1,0,0.0,Alabama,2020-02-02
2,1,0,0.0,Alabama,2020-02-03
3,1,0,0.0,Alabama,2020-02-04
4,1,0,0.0,Alabama,2020-02-05


## Load Small Business revenu data

In [4]:
csv_file = "Resources/state SB_data.csv"
SB_data = pd.read_csv(csv_file)
SB_data.rename(columns={"state_name":"states","date": "dates"}, inplace =True)
SB_data.head()

,dates,statefips,states,revenue_all,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70
0,2020-02-01,1,Alabama,-0.02110,-0.02430,-0.00398,-0.0258,-0.00287
1,2020-02-02,1,Alabama,-0.02320,-0.01600,0.00171,-0.0222,-0.00414
2,2020-02-03,1,Alabama,0.00782,0.00641,-0.01660,0.0536,0.00691
3,2020-02-04,1,Alabama,0.00456,0.00157,-0.04530,0.0687,0.00734
4,2020-02-05,1,Alabama,-0.03180,-0.03050,-0.05070,-0.0164,-0.01190


## Load file with covid deaths

In [5]:
csv_file = "Resources/covid_us_states.csv"
covid_deaths = pd.read_csv(csv_file)
covid_deaths.rename(columns={"state":"states","date": "dates"}, inplace =True)
covid_deaths.sort_values(['states'], inplace = True)
covid_deaths

,Unnamed: 0,dates,states,fips,cases,deaths
0,4579,2020-05-25,Alabama,1,14986,566
93,5129,2020-06-04,Alabama,1,19072,653
94,2324,2020-04-14,Alabama,1,3953,114
95,3699,2020-05-09,Alabama,1,9668,390
96,5294,2020-06-07,Alabama,1,20500,692
...,...,...,...,...,...,...
8326,4028,2020-05-14,Wyoming,56,701,7
8327,4798,2020-05-28,Wyoming,56,876,15
8328,4743,2020-05-27,Wyoming,56,860,14
8330,4688,2020-05-26,Wyoming,56,850,13


In [6]:
## Merging Mobility and Covid Cases and covid deaths

In [7]:
Mobility_and_Covid = pd.merge(data_us, daily_covid, how = "left", on=["dates", "states"])
Mobility_Covid_SB_revenue = pd.merge(Mobility_and_Covid, SB_data, how = "left", on=["dates", "states"])
Mobility_Covid_SB_revenue.drop(["Unnamed: 0","statefips_x","statefips_y","revenue_ss70","revenue_ss65","revenue_ss40"], axis = 1, inplace = True)
Mobility_Covid_SB_revenue_and_Deaths = pd.merge(Mobility_Covid_SB_revenue, covid_deaths, how = "left", on=["dates", "states"])
Mobility_Covid_SB_revenue_and_Deaths.drop(["Unnamed: 0","fips", "cases"], axis = 1, inplace = True)
Mobility_Covid_SB_revenue_and_Deaths

,states,dates,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential,SMA_retail_recreation,SMA_grocery_pharmacy,SMA_parks,SMA_transit,SMA_workplaces,SMA_residential,case_count,new_case_count,revenue_all,revenue_ss60,deaths
0,Alabama,2020-02-15,5.155172,0.230769,49.571429,5.761905,0.784314,-0.863636,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667,0.0,0.0,0.0647,0.00421,NaN
1,Alabama,2020-02-16,-1.454545,-4.015625,-5.476190,2.136364,-1.920000,0.523810,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333,0.0,0.0,0.0427,-0.00607,NaN
2,Alabama,2020-02-17,-1.169492,-4.476190,12.933333,5.727273,-16.264706,4.333333,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667,0.0,0.0,-0.0220,-0.07060,NaN
3,Alabama,2020-02-18,-4.280702,-5.000000,-12.466667,-0.681818,0.338235,1.435897,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667,0.0,0.0,0.0392,-0.02160,NaN
4,Alabama,2020-02-19,1.438596,-0.730159,6.666667,3.714286,1.397059,0.000000,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667,0.0,0.0,0.0347,-0.01220,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8767,Wyoming,2020-07-31,16.000000,28.333333,132.000000,44.666667,-24.190476,2.500000,19.620000,18.425000,NaN,35.429167,-22.344444,3.244444,2726.0,45.9,-0.2040,-0.05510,26.0
8768,Wyoming,2020-08-01,12.363636,17.200000,333.000000,40.888889,-9.933333,-1.000000,18.860000,17.600000,NaN,35.379167,-21.597778,3.122222,2769.0,46.1,-0.2090,-0.05220,26.0
8769,Wyoming,2020-08-02,24.300000,18.750000,321.000000,33.750000,-11.461538,-1.666667,19.056667,17.066667,NaN,34.629167,-20.858974,2.788889,2808.0,47.6,-0.2210,-0.11100,26.0
8770,Wyoming,2020-08-03,29.900000,20.200000,323.000000,34.555556,-23.142857,2.555556,20.236667,17.200000,NaN,34.991667,-21.266667,2.977778,2848.0,46.9,-0.2070,-0.09200,27.0


In [8]:
# Remove uneccessary columns
covid_trends = Mobility_Covid_SB_revenue_and_Deaths [["states", "dates","SMA_retail_recreation", "SMA_grocery_pharmacy", "SMA_parks","SMA_transit","SMA_workplaces","SMA_residential","case_count", "new_case_count","revenue_all","revenue_ss60","deaths"]]
covid_trends.head()

,states,dates,SMA_retail_recreation,SMA_grocery_pharmacy,SMA_parks,SMA_transit,SMA_workplaces,SMA_residential,case_count,new_case_count,revenue_all,revenue_ss60,deaths
0,Alabama,2020-02-15,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667,0.0,0.0,0.0647,0.00421,NaN
1,Alabama,2020-02-16,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333,0.0,0.0,0.0427,-0.00607,NaN
2,Alabama,2020-02-17,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667,0.0,0.0,-0.0220,-0.07060,NaN
3,Alabama,2020-02-18,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667,0.0,0.0,0.0392,-0.02160,NaN
4,Alabama,2020-02-19,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667,0.0,0.0,0.0347,-0.01220,NaN


In [9]:
covid_trends = covid_trends.rename(columns = {"state":'states',"dates":'dates',"SMA_retail_recreation":"retail_recreation", "SMA_grocery_pharmacy":"grocery_pharmacy", "SMA_parks":"parks","SMA_transit":"transit","SMA_workplaces":"workplaces","SMA_residential":"residential","case_count":"case_count", "new_case_count":"new_case_count","revenue_all":"revenue_all","revenue_ss60":"revenue_ss60","deaths":'deaths'})
covid_trends.head()

,states,dates,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential,case_count,new_case_count,revenue_all,revenue_ss60,deaths
0,Alabama,2020-02-15,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667,0.0,0.0,0.0647,0.00421,NaN
1,Alabama,2020-02-16,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333,0.0,0.0,0.0427,-0.00607,NaN
2,Alabama,2020-02-17,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667,0.0,0.0,-0.0220,-0.07060,NaN
3,Alabama,2020-02-18,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667,0.0,0.0,0.0392,-0.02160,NaN
4,Alabama,2020-02-19,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667,0.0,0.0,0.0347,-0.01220,NaN


In [10]:
# Export cleaned df
covid_trends.to_csv("Resources/covid_trends.csv")
covid_trends.to_json(r"Resources/covid_trends.json")

In [11]:
Mobility_Covid_SB_revenue_and_Deaths.to_csv("Resources/Merged_data.csv")
Mobility_Covid_SB_revenue_and_Deaths.to_json(r"Resources/Merged_data.json", orient="records")

In [12]:
Mobility_Covid_SB_revenue_and_Deaths

,states,dates,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential,SMA_retail_recreation,SMA_grocery_pharmacy,SMA_parks,SMA_transit,SMA_workplaces,SMA_residential,case_count,new_case_count,revenue_all,revenue_ss60,deaths
0,Alabama,2020-02-15,5.155172,0.230769,49.571429,5.761905,0.784314,-0.863636,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667,0.0,0.0,0.0647,0.00421,NaN
1,Alabama,2020-02-16,-1.454545,-4.015625,-5.476190,2.136364,-1.920000,0.523810,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333,0.0,0.0,0.0427,-0.00607,NaN
2,Alabama,2020-02-17,-1.169492,-4.476190,12.933333,5.727273,-16.264706,4.333333,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667,0.0,0.0,-0.0220,-0.07060,NaN
3,Alabama,2020-02-18,-4.280702,-5.000000,-12.466667,-0.681818,0.338235,1.435897,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667,0.0,0.0,0.0392,-0.02160,NaN
4,Alabama,2020-02-19,1.438596,-0.730159,6.666667,3.714286,1.397059,0.000000,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667,0.0,0.0,0.0347,-0.01220,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8767,Wyoming,2020-07-31,16.000000,28.333333,132.000000,44.666667,-24.190476,2.500000,19.620000,18.425000,NaN,35.429167,-22.344444,3.244444,2726.0,45.9,-0.2040,-0.05510,26.0
8768,Wyoming,2020-08-01,12.363636,17.200000,333.000000,40.888889,-9.933333,-1.000000,18.860000,17.600000,NaN,35.379167,-21.597778,3.122222,2769.0,46.1,-0.2090,-0.05220,26.0
8769,Wyoming,2020-08-02,24.300000,18.750000,321.000000,33.750000,-11.461538,-1.666667,19.056667,17.066667,NaN,34.629167,-20.858974,2.788889,2808.0,47.6,-0.2210,-0.11100,26.0
8770,Wyoming,2020-08-03,29.900000,20.200000,323.000000,34.555556,-23.142857,2.555556,20.236667,17.200000,NaN,34.991667,-21.266667,2.977778,2848.0,46.9,-0.2070,-0.09200,27.0


In [13]:
covid_trends

,states,dates,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential,case_count,new_case_count,revenue_all,revenue_ss60,deaths
0,Alabama,2020-02-15,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667,0.0,0.0,0.0647,0.00421,NaN
1,Alabama,2020-02-16,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333,0.0,0.0,0.0427,-0.00607,NaN
2,Alabama,2020-02-17,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667,0.0,0.0,-0.0220,-0.07060,NaN
3,Alabama,2020-02-18,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667,0.0,0.0,0.0392,-0.02160,NaN
4,Alabama,2020-02-19,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667,0.0,0.0,0.0347,-0.01220,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8767,Wyoming,2020-07-31,19.620000,18.425000,NaN,35.429167,-22.344444,3.244444,2726.0,45.9,-0.2040,-0.05510,26.0
8768,Wyoming,2020-08-01,18.860000,17.600000,NaN,35.379167,-21.597778,3.122222,2769.0,46.1,-0.2090,-0.05220,26.0
8769,Wyoming,2020-08-02,19.056667,17.066667,NaN,34.629167,-20.858974,2.788889,2808.0,47.6,-0.2210,-0.11100,26.0
8770,Wyoming,2020-08-03,20.236667,17.200000,NaN,34.991667,-21.266667,2.977778,2848.0,46.9,-0.2070,-0.09200,27.0
